# Setup

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
import numpy as np

ModuleNotFoundError: No module named 'open3d'

In [2]:
# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'  #to help with memory segmentation

#Setup device aagnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

## Utils: From script/common.py

In [3]:
from torch.autograd import Variable

class switch(object):
    def __init__(self, value):
        self.value = value
        self.fall = False

    def __iter__(self):
        """Return the match method once, then stop"""
        yield self.match
        raise StopIteration

    def match(self, *args):
        """Indicate whether or not to enter a case suite"""
        if self.fall or not args:
            return True
        elif self.value in args:  # changed for v1.5, see below
            self.fall = True
            return True
        else:
            return False
def s2_grid(n_alpha, n_beta):
    '''
    :return: rings around the equator
    size of the kernel = n_alpha * n_beta
    '''
    beta = np.linspace(start=0, stop=np.pi, num=n_beta, endpoint=False) + np.pi / n_beta / 2
    # ele = np.arcsin(np.linspace(start=0, stop=1, num=n_beta / 2, endpoint=False) + 1 / n_beta / 4)
    # beta = np.concatenate([np.sort(-ele), ele])
    alpha = np.linspace(start=0, stop=2 * np.pi, num=n_alpha, endpoint=False) + np.pi / n_alpha
    B, A = np.meshgrid(beta, alpha, indexing='ij')
    B = B.flatten()
    A = A.flatten()
    grid = np.stack((B, A), axis=1)
    return grid

def change_coordinates(coords, radius, p_from='C', p_to='S'):
    """
    Change Spherical to Cartesian coordinates and vice versa, for points x in S^2.

    In the spherical system, we have coordinates beta and alpha,
    where beta in [0, pi] and alpha in [0, 2pi]

    We use the names beta and alpha for compatibility with the SO(3) code (S^2 being a quotient SO(3)/SO(2)).
    Many sources, like wikipedia use theta=beta and phi=alpha.

    :param coords: coordinate array
    :param p_from: 'C' for Cartesian or 'S' for spherical coordinates
    :param p_to: 'C' for Cartesian or 'S' for spherical coordinates
    :return: new coordinates
    """
    if p_from == p_to:
        return coords
    elif p_from == 'S' and p_to == 'C':

        beta = coords[..., 0]
        alpha = coords[..., 1]
        r = radius

        out = np.empty(beta.shape + (3,))

        ct = np.cos(beta)
        cp = np.cos(alpha)
        st = np.sin(beta)
        sp = np.sin(alpha)
        out[..., 0] = r * st * cp  # x
        out[..., 1] = r * st * sp  # y
        out[..., 2] = r * ct  # z
        return out

    elif p_from == 'C' and p_to == 'S':

        x = coords[..., 0]
        y = coords[..., 1]
        z = coords[..., 2]

        out = np.empty(x.shape + (2,))
        out[..., 0] = np.arccos(z)  # beta
        out[..., 1] = np.arctan2(y, x)  # alpha
        return out

    else:
        raise ValueError('Unknown conversion:' + str(p_from) + ' to ' + str(p_to))

def get_voxel_coordinate(radius, rad_n, azi_n, ele_n):
    grid = s2_grid(n_alpha=azi_n, n_beta=ele_n)
    pts_xyz_on_S2 = change_coordinates(grid, radius, 'S', 'C')
    pts_xyz_on_S2 = np.expand_dims(pts_xyz_on_S2, axis=0).repeat(rad_n, axis=0)
    scale = np.reshape(np.arange(rad_n) / rad_n + 1 / (2 * rad_n), [rad_n, 1, 1])
    pts_xyz = scale * pts_xyz_on_S2
    return pts_xyz

def angles2rotation_matrix(angles):
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(angles[0]), -np.sin(angles[0])],
                   [0, np.sin(angles[0]), np.cos(angles[0])]])
    Ry = np.array([[np.cos(angles[1]), 0, np.sin(angles[1])],
                   [0, 1, 0],
                   [-np.sin(angles[1]), 0, np.cos(angles[1])]])
    Rz = np.array([[np.cos(angles[2]), -np.sin(angles[2]), 0],
                   [np.sin(angles[2]), np.cos(angles[2]), 0],
                   [0, 0, 1]])
    R = np.dot(Rz, np.dot(Ry, Rx))
    return R

def pad_image(input, kernel_size):
    """
    Circularly padding image for convolution
    :param input: [B, C, H, W]
    :param kernel_size:
    :return:
    """
    device = input.device
    if kernel_size % 2 == 0:
        pad_size = kernel_size // 2
        output = torch.cat([input, input[:, :, :, 0:pad_size]], dim=3)
        zeros_pad = torch.zeros([output.shape[0], output.shape[1], pad_size, output.shape[3]]).to(device)
        output = torch.cat([output, zeros_pad], dim=2)
    else:
        pad_size = (kernel_size - 1) // 2
        output = torch.cat([input, input[:, :, :, 0:pad_size]], dim=3)
        output = torch.cat([input[:, :, :, -pad_size:], output], dim=3)
        zeros_pad = torch.zeros([output.shape[0], output.shape[1], pad_size, output.shape[3]]).to(device)
        output = torch.cat([output, zeros_pad], dim=2)
        output = torch.cat([zeros_pad, output], dim=2)
    return output

def pad_image_3d(input, kernel_size):
    """
    Circularly padding image for convolution
    :param input: [B, C, D, H, W]
    :param kernel_size:
    :return:
    """
    device = input.device
    if kernel_size % 2 == 0:
        pad_size = kernel_size // 2
        output = torch.cat([input, input[:, :, :, :, 0:pad_size]], dim=4)
        zeros_pad = torch.zeros([output.shape[0], output.shape[1], output.shape[2], pad_size, output.shape[4]]).to(
            device)
        output = torch.cat([output, zeros_pad], dim=3)
    else:
        pad_size = (kernel_size - 1) // 2
        output = torch.cat([input, input[:, :, :, :, 0:pad_size]], dim=4)
        output = torch.cat([input[:, :, :, :, -pad_size:], output], dim=4)
        zeros_pad = torch.zeros([output.shape[0], output.shape[1], output.shape[2], pad_size, output.shape[4]]).to(
            device)
        output = torch.cat([output, zeros_pad], dim=3)
        output = torch.cat([zeros_pad, output], dim=3)
    return output

def var_to_invar(pts, rad_n, azi_n, ele_n):
    """
    :param pts: input points data, [B, N, nsample, 3]
    :param rad_n: radial number
    :param azi_n: azimuth number
    :param ele_n: elevator number
    :return:
    """
    device = pts.device
    B, N, nsample, C = pts.shape
    assert N == rad_n * azi_n * ele_n
    angle_step = np.array([0, 0, 2 * np.pi / azi_n])
    pts = pts.view(B, rad_n, ele_n, azi_n, nsample, C)

    R = np.zeros([azi_n, 3, 3])
    for i in range(azi_n):
        angle = -1 * i * angle_step
        r = angles2rotation_matrix(angle)
        R[i] = r
    R = torch.FloatTensor(R).to(device)
    R = R.view(1, 1, 1, azi_n, 3, 3).repeat(B, rad_n, ele_n, 1, 1, 1)
    new_pts = torch.matmul(pts, R.transpose(-1, -2))

    del R
    del pts

    return new_pts.view(B, -1, nsample, C)

import numpy as np
from scipy.spatial import cKDTree

def ball_query(pts, new_pts, radius, nsample):
    """
    :param pts: all points, [B, N, 3]
    :param new_pts: query points, [B, S, 3]
    :param radius: local spherical radius
    :param nsample: max sample number in local sphere
    :return: indices of sampled points around new_pts [B, S, nsample]
    """
    device = pts.device
    B, N, C = pts.shape
    _, S, _ = new_pts.shape

    # Create an empty tensor to hold the indices of sampled points
    sampled_indices = torch.zeros(B, S, nsample, dtype=torch.long, device=device)

    for b in range(B):
        # Calculate pairwise distances between all points and query points
        pts_b = pts[b]  # [N, 3]
        new_pts_b = new_pts[b]  # [S, 3]

        # Expand dimensions for broadcasting
        pts_exp = pts_b.unsqueeze(0)  # [1, N, 3]
        new_pts_exp = new_pts_b.unsqueeze(1)  # [S, 1, 3]

        # Compute squared distances
        dists_sq = torch.sum((pts_exp - new_pts_exp) ** 2, dim=-1)  # [S, N]

        # Find points within the radius
        mask = dists_sq <= radius ** 2
        for s in range(S):
            indices = torch.nonzero(mask[s]).squeeze(1)  # [num_points_in_sphere]

            if indices.numel() > nsample:
                # If there are more points than nsample, randomly sample
                indices = indices[torch.randperm(indices.size(0))[:nsample]]
            elif indices.numel() < nsample:
                # If there are fewer points than nsample, pad with zeros if indices is empty
                pad_size = nsample - indices.numel()
                if pad_size > 0:
                    if indices.numel() == 0:
                        # No points found, pad with zeros (or any other placeholder index like -1)
                        indices = torch.zeros(nsample, dtype=torch.long, device=device)
                    else:
                        # Repeat the last index to fill remaining spots
                        indices = torch.cat([indices, indices[-1].repeat(pad_size)])

            # Store the indices of the sampled points
            sampled_indices[b, s, :indices.numel()] = indices

    return sampled_indices

def grouping_operation(features, idx):
    """
    Agrupa características basadas en índices.

    :param features: Tensor de características con forma (B, C, N)
    :param idx: Tensor de índices con forma (B, npoint, nsample)
    :return: Tensor de características agrupadas con forma (B, C, npoint, nsample)
    """
    B, C, N = features.shape
    _, npoint, nsample = idx.shape

    # Expande 'idx' para que coincida con la forma de 'features'
    idx_expanded = idx.unsqueeze(1)  # Forma (B, 1, npoint, nsample)

    # Ajustar 'features' para usar 'torch.gather'
    features_expanded = features.unsqueeze(2).expand(-1, -1, npoint, -1)  # Forma (B, C, npoint, N)

    # Agrupa características usando los índices expandidos
    grouped_features = torch.gather(features_expanded, 3, idx_expanded)

    return grouped_features

def sphere_query(pts, new_pts, radius, nsample):
  """
  :param pts: all points, [B. N. 3]
  :param new_pts: query points, [B, S. 3]
  :param radius: local sperical radius
  :param nsample: max sample number in local sphere
  :return:
  """

  device = pts.device
  B, N, C = pts.shape
  _, S, _ = new_pts.shape

  pts = pts.contiguous()
  new_pts = new_pts.contiguous()
  group_idx = ball_query(pts, new_pts, radius, nsample)
  #print(group_idx)
  #group_idx = pnt2.ball_query(radius, nsample, pts, new_pts)
  mask = group_idx[:, :, 0].unsqueeze(2).repeat(1, 1, nsample)
  mask = (group_idx == mask).float()
  mask[:, :, 0] = 0

  # C implementation
  pts_trans = pts.transpose(1, 2).contiguous()
  #new_points = pnt2.grouping_operation(pts_trans, group_idx)  # (B, 3, npoint, nsample)
  new_points = grouping_operation(pts_trans, group_idx)  # (B, 3, npoint, nsample)
  new_points = new_points.permute([0, 2, 3, 1])

  # replace the wrong points using new_pts
  mask = mask.unsqueeze(3).repeat([1, 1, 1, 3])
  # new_pts = new_pts.unsqueeze(2).repeat([1, 1, nsample + 1, 1])
  new_pts = new_pts.unsqueeze(2).repeat([1, 1, nsample, 1])
  n_points = new_points * (1 - mask).float() + new_pts * mask.float()

  del mask
  del new_points
  del group_idx
  del new_pts
  del pts
  del pts_trans

  return n_points

def l2_norm(input, axis=1):
    norm = torch.norm(input, p=2, dim=axis, keepdim=True)
    output = torch.div(input, norm)
    return output

def cal_Z_axis(local_cor, local_weight=None, ref_point=None):
    device = local_cor.device
    B, N, _ = local_cor.shape
    cov_matrix = torch.matmul(local_cor.transpose(-1, -2), local_cor) if local_weight is None \
        else Variable(torch.matmul(local_cor.transpose(-1, -2), local_cor * local_weight), requires_grad=True)
    #Z_axis = torch.symeig(cov_matrix, eigenvectors=True)[1][:, :, 0]
    Z_axis = torch.linalg.eigh(cov_matrix, UPLO="U")[1][:, :, 0]
    mask = (torch.sum(-Z_axis * ref_point, dim=1) < 0).float().unsqueeze(1)
    Z_axis = Z_axis * (1 - mask) - Z_axis * mask

    return Z_axis

def RodsRotatFormula(a, b):
    B, _ = a.shape
    device = a.device
    b = b.to(device)
    c = torch.cross(a, b)
    theta = torch.acos(F.cosine_similarity(a, b)).unsqueeze(1).unsqueeze(2)

    c = F.normalize(c, p=2, dim=1)
    one = torch.ones(B, 1, 1).to(device)
    zero = torch.zeros(B, 1, 1).to(device)
    a11 = zero
    a12 = -c[:, 2].unsqueeze(1).unsqueeze(2)
    a13 = c[:, 1].unsqueeze(1).unsqueeze(2)
    a21 = c[:, 2].unsqueeze(1).unsqueeze(2)
    a22 = zero
    a23 = -c[:, 0].unsqueeze(1).unsqueeze(2)
    a31 = -c[:, 1].unsqueeze(1).unsqueeze(2)
    a32 = c[:, 0].unsqueeze(1).unsqueeze(2)
    a33 = zero
    Rx = torch.cat(
        (torch.cat((a11, a12, a13), dim=2), torch.cat((a21, a22, a23), dim=2), torch.cat((a31, a32, a33), dim=2)),
        dim=1)
    I = torch.eye(3).to(device)
    R = I.unsqueeze(0).repeat(B, 1, 1) + torch.sin(theta) * Rx + (1 - torch.cos(theta)) * torch.matmul(Rx, Rx)
    return R.transpose(-1, -2)

# Model


## From network/threeDCCN

In [4]:
import pdb
import torch
import torch.nn as nn
import torch.nn.functional as F
# import script.common as cm


class BaseNet(nn.Module):
    """ Takes a list of images as input, and returns for each image:
        - a pixelwise descriptor
        - a pixelwise confidence
    """

    def forward_one(self, x):
        raise NotImplementedError()

    def forward(self, imgs):
        res = self.forward_one(imgs)
        return res


class Cyclindrical_ConvNet(BaseNet):
    def __init__(self, inchan=3, dilated=True, dilation=1, bn=True, bn_affine=False):
        BaseNet.__init__(self)
        self.inchan = inchan
        self.curchan = inchan
        self.dilated = dilated
        self.dilation = dilation
        self.bn = bn
        self.bn_affine = bn_affine
        self.ops = nn.ModuleList([])

    def _make_bn_2d(self, outd):
        return nn.BatchNorm2d(outd, affine=self.bn_affine)

    def _make_bn_3d(self, outd):
        return nn.BatchNorm3d(outd, affine=self.bn_affine)

    def _add_conv_2d(self, outd, k=3, stride=1, dilation=1, bn=True, relu=True):
        d = self.dilation * dilation
        self.dilation *= stride
        self.ops.append(nn.Conv2d(self.curchan, outd, kernel_size=(k, k), dilation=d))
        if bn and self.bn: self.ops.append(self._make_bn_2d(outd))
        if relu: self.ops.append(nn.ReLU(inplace=True))
        self.curchan = outd

    def _add_conv_3d(self, outd, k, stride=1, dilation=1, bn=True, relu=True):
        d = self.dilation * dilation
        self.dilation *= stride
        self.ops.append(nn.Conv3d(self.curchan, outd, kernel_size=(k[0], k[1], k[2]), dilation=d))
        if bn and self.bn: self.ops.append(self._make_bn_3d(outd))
        if relu: self.ops.append(nn.ReLU(inplace=True))
        self.curchan = outd

    def forward_one(self, x):
        assert self.ops, "You need to add convolutions first"
        for n, op in enumerate(self.ops):
            k_exist = hasattr(op, 'kernel_size')
            if k_exist:
                if len(op.kernel_size) == 3:
                    x = pad_image_3d(x, op.kernel_size[1] + (op.kernel_size[1] - 1) * (op.dilation[0] - 1))
                else:
                    if len(x.shape) == 5:
                        x = x.squeeze(2)
                    x = pad_image(x, op.kernel_size[0] + (op.kernel_size[0] - 1) * (op.dilation[0] - 1))
            x = op(x)
        return x


class Cylindrical_Net(Cyclindrical_ConvNet):
    """ Compute a descriptor for all overlapping patches.
        From the L2Net paper (CVPR'17).
    """

    def __init__(self, inchan=16, dim=32, **kw):
        Cyclindrical_ConvNet.__init__(self, inchan=inchan, **kw)
        add_conv_2d = lambda n, **kw: self._add_conv_2d(n, **kw)
        add_conv_3d = lambda n, **kw: self._add_conv_3d(n, **kw)
        add_conv_3d(32, k=[3, 3, 3])
        add_conv_3d(32, k=[3, 3, 3])
        add_conv_3d(64, k=[3, 3, 3])
        add_conv_3d(64, k=[3, 3, 3])
        add_conv_2d(128, stride=2)
        add_conv_2d(128)
        add_conv_2d(64, stride=2)
        add_conv_2d(64)
        add_conv_2d(32, k=2, stride=2, relu=False)
        add_conv_2d(32, k=2, stride=2, relu=False)
        add_conv_2d(dim, k=2, stride=2, bn=False, relu=False)
        self.out_dim = dim

## From network/SpinNet

A descriptor is a representation that captures distinctive features or characteristics of points in a point cloud.

In point cloud registration, descriptors are used to identify and match corresponding points between different point clouds

In deep learning, descriptors are typically learned features that capture high-level patterns or representations from raw point cloud data. These descriptors are often produced by neural network models and can be used for various tasks such as classification, segmentation, and registration.

In [5]:
class Descriptor_Net(nn.Module):
    def __init__(self, des_r, rad_n, azi_n, ele_n, voxel_r, voxel_sample, dataset):
        super(Descriptor_Net, self).__init__()
        self.des_r = des_r
        self.rad_n = rad_n
        self.azi_n = azi_n
        self.ele_n = ele_n
        self.voxel_r = voxel_r
        self.voxel_sample = voxel_sample
        self.dataset = dataset

        self.bn_xyz_raising = nn.BatchNorm2d(16)
        self.bn_mapping = nn.BatchNorm2d(16)
        self.activation = nn.ReLU()
        self.xyz_raising = nn.Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
        self.conv_net = Cylindrical_Net(inchan=16, dim=32)

    def forward(self, input):
        center = input[:, -1, :].unsqueeze(1)
        delta_x = input[:, :, 0:3] - center[:, :, 0:3]  # (B, npoint, 3), normalized coordinates
        for case in switch(self.dataset):
            if case('3DMatch'):
                z_axis = cal_Z_axis(delta_x, ref_point=input[:, -1, :3])
                z_axis = l2_norm(z_axis, axis=1)
                R = RodsRotatFormula(z_axis, torch.FloatTensor([0, 0, 1]).unsqueeze(0).repeat(z_axis.shape[0], 1))
                delta_x = torch.matmul(delta_x, R)
                break
            if case('KITTI'):
                break

        # partition the local surface along elevator, azimuth, radial dimensions
        S2_xyz = torch.FloatTensor(get_voxel_coordinate(radius=self.des_r,
                                                           rad_n=self.rad_n,
                                                           azi_n=self.azi_n,
                                                           ele_n=self.ele_n))

        pts_xyz = S2_xyz.view(1, -1, 3).repeat([delta_x.shape[0], 1, 1]).to(device)  #.cuda()
        # query points in sphere
        new_points = sphere_query(delta_x, pts_xyz, radius=self.voxel_r,
                                     nsample=self.voxel_sample)
        # transform rotation-variant coords into rotation-invariant coords
        new_points = new_points - pts_xyz.unsqueeze(2).repeat([1, 1, self.voxel_sample, 1])
        new_points = var_to_invar(new_points, self.rad_n, self.azi_n, self.ele_n)

        new_points = new_points.permute(0, 3, 1, 2)  # (B, C_in, npoint, nsample), input features
        C_in = new_points.size()[1]
        nsample = new_points.size()[3]
        x = self.activation(self.bn_xyz_raising(self.xyz_raising(new_points)))
        x = F.max_pool2d(x, kernel_size=(1, nsample)).squeeze(3)  # (B, C_in, npoint)
        del new_points
        del pts_xyz
        x = x.view(x.shape[0], x.shape[1], self.rad_n, self.ele_n, self.azi_n)

        x = self.conv_net(x)
        x = F.max_pool2d(x, kernel_size=(x.shape[2], x.shape[3]))

        return x

    def get_parameter(self):
        return list(self.parameters())

# Load pretrained model

In [6]:
model = Descriptor_Net(0.30, 9, 80, 40, 0.04, 30, '3DMatch')
model = nn.DataParallel(model, device_ids=[0])

In [8]:
filename = "./PreTrainedModels/3DMatch_best.pkl"
model.load_state_dict(torch.load(filename, weights_only=False, map_location=device))

<All keys matched successfully>

In [9]:
model

DataParallel(
  (module): Descriptor_Net(
    (bn_xyz_raising): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn_mapping): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
    (xyz_raising): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
    (conv_net): Cylindrical_Net(
      (ops): ModuleList(
        (0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
        (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
        (4): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
        (7): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        (8): ReLU(inplace=True)
        (9): Conv3d(64

# Getting Data Ready


## Import Data: From 3DMatch/intermediate-files-real/7-scenes-redkitchen

Extract tar file, only using 1 scene: 7-scenes-redkitchen

## From ThreeDMatch/Test/tools.py

In [10]:
import os

def get_pcd(pcdpath, filename):
    return o3d.io.read_point_cloud(os.path.join(pcdpath, filename + '.ply'))

def get_keypts(keyptspath, filename):
    keypts = np.fromfile(os.path.join(keyptspath, filename + '.keypts.bin'), dtype=np.float32)
    num_keypts = int(keypts[0])
    keypts = keypts[1:].reshape([num_keypts, 3])
    return keypts

## From ThreeDMatch/Test/preparation.py

In [18]:
import open3d
import time
from sklearn.neighbors import KDTree

def make_open3d_point_cloud(xyz, color=None):
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(xyz)
    if color is not None:
        pcd.paint_uniform_color(color)
    return pcd

def build_patch_input(pcd, keypts, vicinity=0.3, num_points_per_patch=2048):
    refer_pts = keypts.astype(np.float32)
    pts = np.array(pcd.points).astype(np.float32)
    num_patches = refer_pts.shape[0]
    tree = KDTree(pts[:, 0:3])
    ind_local = tree.query_radius(refer_pts[:, 0:3], r=vicinity)
    local_patches = np.zeros([num_patches, num_points_per_patch, 3], dtype=float)
    for i in range(num_patches):
        local_neighbors = pts[ind_local[i], :]
        if local_neighbors.shape[0] >= num_points_per_patch:
            temp = np.random.choice(range(local_neighbors.shape[0]), num_points_per_patch, replace=False)
            local_neighbors = local_neighbors[temp]
            local_neighbors[-1, :] = refer_pts[i, :]
        else:
            fix_idx = np.asarray(range(local_neighbors.shape[0]))
            while local_neighbors.shape[0] + fix_idx.shape[0] < num_points_per_patch:
                fix_idx = np.concatenate((fix_idx, np.asarray(range(local_neighbors.shape[0]))), axis=0)
            random_idx = np.random.choice(local_neighbors.shape[0], num_points_per_patch - fix_idx.shape[0],
                                          replace=False)
            choice_idx = np.concatenate((fix_idx, random_idx), axis=0)
            local_neighbors = local_neighbors[choice_idx]
            local_neighbors[-1, :] = refer_pts[i, :]
        local_patches[i] = local_neighbors

    return local_patches

def prepare_patch(pcdpath, filename, keyptspath, trans_matrix):
    pcd = get_pcd(pcdpath, filename)
    keypts = get_keypts(keyptspath, filename)
    # load D3Feat keypts
    if is_D3Feat_keypts:
        keypts_path = './D3Feat_contralo-54-pred/keypoints/' + pcdpath.split('/')[-2] + '/' + filename + '.npy'
        keypts = np.load(keypts_path)
        keypts = keypts[-5000:, :]
    if is_rotate_dataset:
        # Add arbitrary rotation
        # rotate terminal frament with an arbitrary angle around the z-axis
        angles_3d = np.random.rand(3) * np.pi * 2
        R = angles2rotation_matrix(angles_3d)
        T = np.identity(4)
        T[:3, :3] = R
        pcd.transform(T)
        keypts_pcd = make_open3d_point_cloud(keypts)
        keypts_pcd.transform(T)
        keypts = np.array(keypts_pcd.points)
        trans_matrix.append(T)

    local_patches = build_patch_input(pcd, keypts)  # [num_keypts, 1024, 4]
    return local_patches

def generate_descriptor(model, desc_name, pcdpath, keyptspath, descpath):
  model.eval()
  num_frag = len(os.listdir(pcdpath))
  num_desc = len(os.listdir(descpath))
  trans_matrix = []
  if num_frag == num_desc:
    print("Descriptor already prepared.")
    return
  for j in range(num_frag):
      local_patches = prepare_patch(pcdpath, 'cloud_bin_' + str(j), keyptspath, trans_matrix)
      input_ = torch.tensor(local_patches.astype(np.float32)).to(device)
      B = input_.shape[0]
      input_ = input_.to(device)
      model = model.to(device)
      #input_ = input_.cuda()
      #model = model.cuda()
      # calculate descriptors
      desc_list = []
      start_time = time.time()
      desc_len = 32
      step_size = 10 #100
      iter_num = int(np.ceil(B / step_size))
      for k in range(iter_num):
          if k %10 == 0:
            print(f"{k}/{iter_num}")
          if k == iter_num - 1:
              desc = model(input_[k * step_size:, :, :])
          else:
              desc = model(input_[k * step_size: (k + 1) * step_size, :, :])
          if desc == None:
              print("se despichó Tere", k)
          desc_list.append(desc.view(desc.shape[0], desc_len).detach().cpu().numpy())
          del desc
      step_time = time.time() - start_time
      print(f'Finish {B} descriptors spend {step_time:.4f}s')
      #print(descpath + 'cloud_bin_' + str(j) + f".desc.{desc_name}.bin") #!Extra
      desc = np.concatenate(desc_list, 0).reshape([B, desc_len])
      np.save(descpath + 'cloud_bin_' + str(j) + f".desc.{desc_name}.bin", desc.astype(np.float32))
  if is_rotate_dataset:
      scene_name = pcdpath.split('/')[-2]
      all_trans_matrix[scene_name] = trans_matrix

In [19]:
import os
import time

experiment_id = time.strftime('%m%d%H%M')
model_str = experiment_id
if not os.path.exists(f"SpinNet_desc_{model_str}/"):
        os.mkdir(f"SpinNet_desc_{model_str}")

scene_list = ['7-scenes-redkitchen']

all_trans_matrix = {}
is_rotate_dataset = False
is_D3Feat_keypts = False
for scene in scene_list:
  pcdpath = f'./Data/3DMatch/fragments/{scene}/'
  interpath = f'./Data/3DMatch/intermediate-files-real/{scene}/'
  keyptspath = interpath
  descpath = os.path.join("", f"SpinNet_desc_{model_str}/{scene}/")
  if not os.path.exists(descpath):
    os.makedirs(descpath)

  start_time = time.time()
  print(f"Begin Processing {scene}")
  #torch.cuda.empty_cache()
  generate_descriptor(model, desc_name='SpinNet', pcdpath=pcdpath, keyptspath=keyptspath, descpath=descpath)
  print(f"Finish in {time.time() - start_time}s")
if is_rotate_dataset:
    np.save(f"trans_matrix", all_trans_matrix)

Begin Processing 7-scenes-redkitchen
0/500
Finish 5000 descriptors spend 10.3766s
SpinNet_desc_09130852/7-scenes-redkitchen/cloud_bin_0.desc.SpinNet.bin


ValueError: cannot reshape array of size 320 into shape (5000,32)

Acabó pero no se guardó el archivo, tampoco tiró error
revisar si el np.save está sirviendo

In [15]:
import numpy as np
desc = np.array([1,2,3])
np.save("SpinNet_desc_09130852/7-scenes-redkitchen/cloud_bin_0.desc.SpinNet.bin", desc)